<a href="https://colab.research.google.com/github/CP2J/cp2j/blob/ACJ-17-knn-cf/cf_knn_230405.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 협업필터링 + KNN

In [ ]:
import pandas as pd
import numpy as np
import warnings
# 경고 제거
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 전체 데이터 다시 로딩
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', sep = '\t', names=r_cols, header = None)
ratings = ratings.drop('timestamp', axis=1) # timestamp 지우기

# Rating df의 user_id를 타겟(종속변수, 예측값)으로 train, test stratified split 실시(user_id 기준)
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=12)

# "유저별 영화 평점"을 알아보기 위해 pivot table 활용, 결측치 처리
# train 셋을 full matrix로 변환
rating_matrix = x_train.pivot_table(values = 'rating', index = 'user_id', columns = 'movie_id')
rating_matrix = rating_matrix.fillna(0)

# 유저들의 영화 평점 데이터프레임의 코사인유사도를 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

In [ ]:
# RMSE 계산함수 (오차(true-pred) 제곱 평균의 제곱근)
def RMSE(y_true, y_pred):
    return np.sqrt( np.mean(( np.array(y_true) - np.array(y_pred) )**2 ) )

def score_2(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id']) # 테스트로 해야되나?
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# CF + KNN
def cf_knn(user_id, movie_id, neighbor_size=10):
    # checks if the column with the name "movie_id" is present in the DataFrame "rating_matrix"
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기 (총 943개) 
        sim_scores = user_similarity[user_id]
        
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        this_movie_ratings = rating_matrix[movie_id]
        
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = rating_matrix[movie_id][rating_matrix[movie_id].isnull()].index
        
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        this_movie_ratings = this_movie_ratings.dropna()
        
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거 -> 평가한 사람들만의 유사도 점수가 됨
        sim_scores = sim_scores.drop(none_rating_idx)

        ### KNN 추가 부분
        if neighbor_size == 0:  # neighbor_size 가 지정되지 않은 경우
            # 현재 영화를 평가한 모든 사용자의 가중평균 구하기
            mean_rating = np.dot(sim_scores, this_movie_ratings) / sim_scores.sum()

        else:                   # neighbor_size 가 지정된 경우
            # 지정된 neighbor_size와 해당 영화를 평가한 총 사용자수 중 작은 것으로 neighbor_size 결정
            neighbor_size = min( neighbor_size, len(sim_scores) )

            # array로 바꾸기 - argsort 를 사용하기 위해
            sim_scores = np.array(sim_scores)
            movie_ratings = np.array(rating_matrix) # 길이가 1163인 리스트 943개를 원소로 갖는 배열

            # 요소 크기 오름차순 정렬한 인덱스(user_id) 배열 반환 : argsort - Returns: index_array
            # https://codetorial.net/tips_and_examples/numpy_argsort.html
            user_idx = np.argsort(sim_scores)

            # 오름차순 정렬한 User 유사도 배열을 큰 것부터(뒤에서) neighbor_size 만큼 받기
            sim_scores = sim_scores[user_idx][-neighbor_size:]

            # 영화 Rating을 neighbor_size만큼 받기
            movie_ratings = movie_ratings[user_idx][-neighbor_size:]

            # 최종 예측값 계산
            mean_rating = np.dot( sim_scores, movie_ratings ) / sim_scores.sum()
    else:
        # movie_id가 rating_matrix의 칼럼값에 없는 경우
        mean_rating = 3.0
        
    return mean_rating

In [ ]:
score_2(cf_knn, 10)
# neighbor 사이즈가 입력된 경우에도, 해당 영화를 평가한 유저의 수가 K 보다 적을 수 있으므로 한번 더 확인 해야함

In [ ]:
#최적의 K값 확인 
for k in (10,20,30,40,50):
    print('k = ', k,'RMSE = ', score_2(cf_knn, k))

In [ ]:
# 추천하기
def cf_knn_recommender(user, n_items=10, neighbor_size=10):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []

    # 이미 평가한(not null) 영화의 열 인덱스(movie_id) 추출 -> 추천시 제외하기 위해
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index
    items = rating_matrix.loc[user].drop(rated_index)
    
    # 여기서 파라미터 값이 3개 들어감 
    # 예상평점 계산
    for item in items.index:
        predictions.append(cf_knn(user, item, neighbor_size))
        
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]    # 예상평점이 가장 높은 영화 선택
    recommended_items = movies.loc[recommendations.index]['title']
    return recommended_items

# 영화 추천 함수 부르기
cf_knn_recommender(2, 10, 20)